## Predicting the Stock Market 

In this proyect we'll be working with data from the stock market, specifically the S&P 500 Index. The objective in scope is the train a Linear Regression model that is able to predict the price of a given stock with an acceptable error margin. The model is going to be trained on historical data from 1950-2012 and tested on data from 2013-2015. 

Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

The columns of the dataset are:

* Date -- The date of the record.
* Open -- The opening price of the day (when trading starts).
* High -- The highest trade price during the day.
* Low -- The lowest trade price during the day.
* Close -- The closing price for the day (when trading is finished).
* Volume -- The number of shares traded.
* Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read more [here](https://www.kaggle.com/samaxtech/sp500-index-data/data).

In [17]:
import pandas as pd
from datetime import datetime

data = pd.read_csv('sphist.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


## Feature Engineering 

We'll now create new columns with data from previous row. We have to be careful not to inyect future data in the current row which could lead to biased predictions and overfitting. We will use the rolling and shift methods in pandas to average the closing price of the market for diffrent time frames. We will create new features containing information of the mean and standard deviation of the last 5 and 30 days.

In [18]:
data['five_days_mean'] = data['Adj Close'].rolling(window=5).mean()
data['five_days_std'] = data['Adj Close'].rolling(window=5).std()

data['one_year_mean'] = data['Adj Close'].rolling(window=365).mean()
data['one_year_std'] = data['Adj Close'].rolling(window=365).std()

data = data.shift(periods=1)

data.tail()

,Date,Open,High,Low,Close,Volume,Adj Close,five_days_mean,five_days_std,one_year_mean,one_year_std
4,2015-11-30,2090.949951,2093.810059,2080.409912,2080.409912,4.245030e+09,2080.409912,2087.024023,3.916109,2035.531178,64.370261
3,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2090.231982,7.956808,2035.914082,64.352527
2,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2088.306006,9.333599,2036.234356,64.277554
1,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2080.456006,19.599946,2036.507343,64.121622
0,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.771973,19.806136,2036.869425,64.058862


In [22]:
data.dropna(inplace=True)
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,five_days_mean,five_days_std,one_year_mean,one_year_std
16224,1951-06-18,22.049999,22.049999,22.049999,22.049999,1050000.0,22.049999,21.800,0.256223,19.447726,1.790253
16223,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.900,0.213659,19.462411,1.789307
16222,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.972,0.092574,19.476274,1.788613
16221,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.960,0.115108,19.489562,1.787659
16220,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.862,0.204132,19.502082,1.786038


## Split data into Train and Test

Our testing data will be starting from 2013 onward. The remaining data will be our training dataset.

In [26]:
train = data[data['Date'] < '2013-01-01']

test = data[data['Date'] >= '2013-01-01']

,Date,Open,High,Low,Close,Volume,Adj Close,five_days_mean,five_days_std,one_year_mean,one_year_std
737,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1425.793994,22.261321,1327.908247,90.738976
736,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1433.702002,26.274326,1328.224877,90.995857
735,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1443.376001,27.945242,1328.557617,91.279049
734,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1455.267993,16.453319,1328.898603,91.544368
733,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1461.460010,3.505368,1329.241644,91.790064


# Model Training

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

features = ['five_days_mean', 'five_days_std', 'one_year_mean', 'one_year_std']
lr = LinearRegression()
lr.fit(data[features], data['Close'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Model Prediction and Error

In [30]:
predictions = lr.predict(test[features])

In [34]:
mse = mean_squared_error(test['Close'], predictions)
rmse = mse**(1/2)
rmse

16.394293822384196

We succesfully created a model that is able to predict the stock market price with an RMSE error of 16.39 dollars. 